In [1]:
from scipy.sparse import csr_matrix
import sparse
import numpy as np
import pyamg
import scipy.sparse

In [2]:
class mycsr(csr_matrix):
    """
    *OpenMP Enabled*
    """
    def _mul_vector(self, other):        
        M,N = self.shape

        # output array
        result = np.zeros(M, dtype=scipy.sparse.sputils.upcast_char(self.dtype.char,
                                                                    other.dtype.char))

        # csr_matvec or csc_matvec
        sparse.csr_matvec_omp(M, N, self.indptr, self.indices, self.data, other, result)

        return result

mycsr.__doc__ += csr_matrix.__doc__

In [3]:
from timeit import default_timer as timer

In [4]:
A = pyamg.gallery.poisson((3,3), format='csr')
v = np.random.rand(A.shape[0])

In [5]:
Amycsr = mycsr(A)

In [6]:
w1 = A * v

In [7]:
w2 = Amycsr * v

In [8]:
np.testing.assert_array_almost_equal(w1, w2)

In [9]:
Amycsr?

In [21]:
np.random.seed(555)
nx = 400

t0 = timer()
A = pyamg.gallery.poisson((nx, nx, nx), format='csr')
b = np.random.rand(A.shape[0])
x0 = np.random.rand(A.shape[0])
t1 = timer()
print(t1-t0, ' seconds')

In [22]:
t0 = timer()
ml = pyamg.smoothed_aggregation_solver(A, max_coarse=10)
t1 = timer()
print(t1-t0, ' seconds')

In [23]:
t0 = timer()
x = ml.solve(b, x0=x0, maxiter=10)
t1 = timer()
print(t1-t0, ' seconds')

251.45733980834484  seconds


In [24]:
t0 = timer()
for i in range(len(ml.levels)):
    ml.levels[i].A = mycsr(ml.levels[i].A)
t1 = timer()
print(t1-t0, ' seconds')

10.465544525533915  seconds


In [26]:
t0 = timer()
x = ml.solve(b, x0=x0, maxiter=10)
t1 = timer()
print(t1-t0, ' seconds')

95.59064931049943  seconds


In [27]:
ml

multilevel_solver
Number of Levels:     6
Operator Complexity:  1.573
Grid Complexity:      1.123
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0     64000000    447040000 [63.59%]
    1      7731493    244080963 [34.72%]
    2       167960     11786858 [ 1.68%]
    3         1627       108071 [ 0.02%]
    4           21          409 [ 0.00%]
    5            1            1 [ 0.00%]

In [30]:
for i in range(len(ml.levels)):
    A = ml.levels[i].A
    v = np.random.rand(A.shape[0])
    t0 = timer()
    w = A * v
    t1 = timer()
    print(t1 - t0, "seconds")

0.13179339468479156 seconds
0.1051749587059021 seconds
0.0036995112895965576 seconds
8.744373917579651e-05 seconds
3.0178576707839966e-05 seconds
2.7876347303390503e-05 seconds


In [31]:
for i in range(len(ml.levels)):
    t0 = timer()
    B = ml.levels[i].A.copy()
    w = A * v
    t1 = timer()
    print(t1 - t0, "seconds")

2.4414535984396935 seconds
1.9934331364929676 seconds
0.15189167112112045 seconds
0.004616264253854752 seconds
0.004179149866104126 seconds
0.004165146499872208 seconds


In [40]:
t0 = timer()
for i in range(len(ml.levels)):
    ml.levels[i].A = mycsr(ml.levels[i].A)
    if hasattr(ml.levels[i], 'P'):
        ml.levels[i].P = mycsr(ml.levels[i].P)
    if hasattr(ml.levels[i], 'R'):
        ml.levels[i].R = mycsr(ml.levels[i].R)
t1 = timer()
print(t1-t0, ' seconds')

0.001399170607328415  seconds


In [41]:
t0 = timer()
x = ml.solve(b, x0=x0, maxiter=10)
t1 = timer()
print(t1-t0, ' seconds')

82.09879674762487  seconds
